Adaptado de https://www.tensorflow.org/text/tutorials/text_generation.


# Generacion de Textos Con Red Neuronal Recurrente

En esta demostracion utilizaremos textos de Cervantes para entrenar un modelo que sea capaz de completar las frases tal y como lo haria el propio Cervantes, o lo mas parecido posible.

https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79

*Del buen suceso que el valeroso Don Quijote tuvo en la espantable y jamás imaginada aventura de los molinos de viento, con otros sucesos dignos de felice recordación
En esto descubrieron treinta o cuarenta molinos de viento que hay en aquel campo, y así como Don Quijote los vió, dijo a su escudero: la ventura va guiando nuestras cosas mejor de lo que acertáramos a desear; porque ves allí, amigo Sancho Panza, donde se descubren treinta o poco más desaforados gigantes con quien pienso hacer batalla, y quitarles a todos las vidas, con cuyos despojos comenzaremos a enriquecer: que esta es buena guerra, y es gran servicio de Dios quitar tan mala simiente de sobre la faz de la tierra. ¿Qué gigantes? dijo Sancho Panza. Aquellos que allí ves, respondió su amo, de los brazos largos, que los suelen tener algunos de casi dos leguas. Mire vuestra merced, respondió Sancho, que aquellos que allí se parecen no son gigantes, sino molinos de viento, y lo que en ellos parecen brazos son las aspas, que volteadas del viento hacen andar la piedra del molino. Bien parece, respondió Don Quijote, que no estás cursado en esto de las aventuras; ellos son gigantes, y si tienes miedo quítate de ahí, y ponte en oración en el espacio que yo voy a entrar con ellos en fiera y desigual batalla. Y diciendo esto, dio de espuelas a su caballo Rocinante, sin atender a las voces que su escudero Sancho le daba, advirtiéndole que sin duda alguna eran molinos de viento, y no gigantes aquellos que iba a acometer. Pero él iba tan puesto en que eran gigantes, que ni oía las voces de su escudero Sancho, ni echaba de ver, aunque estaba ya bien cerca, lo que eran; antes iba diciendo en voces altas: non fuyades, cobardes y viles criaturas, que un solo caballero es el que os acomete. Levantóse en esto un poco de viento y las grandes aspas comenzaron a moverse, lo cual visto por Don Quijote, dijo: pues aunque mováis más brazos que los del gigante Briareo, me lo habéis de pagar.*

Las frases pueden mantener una cierta estructura gramaticalmente correcta, generalmente el texto es ininteligible y carece de sentido. El modelo no entiende el significado de las palabras. 

El modelo esta basado en caracteres individuales, al inicio del entrenamiento el modelo no sabe que es una letra ni que una palabra forma una unidad semantica.

La estructura es similar a la prosa original de Cervantes.

El modelo es capaz de generar textos largos a partir de secciones o lotes relativamente pequeños.

## Preparacion

Importamos tensorflow y otras librerias:

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

Descargamos nuestro texto de entrenamiento:

In [2]:
url = 'https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt'

In [3]:
archivo_texto = tf.keras.utils.get_file('elquijote.txt', url)

1073152/1060259 [==============================] - 0s 0us/step


In [4]:
archivo_texto

'/root/.keras/datasets/elquijote.txt'

Leemos el contenido del texto:

In [5]:
# Leemos y recodificamos para compatibilidad.
texto = open(archivo_texto, 'rb').read().decode(encoding='utf-8')
# longitud de caracteres
print(f'Longitud del texto: {len(texto)} caracteres.')

Longitud del texto: 1038397 caracteres.


In [6]:
# Miramos 150 caracteres al azar:
import random
cata = 280
N = 5
for i in range(N):
  inicio = random.randint(0, len(texto)-cata)
  print("-"*100)
  print(texto[inicio:inicio+cata])

----------------------------------------------------------------------------------------------------
sase a él y a trescientas cabras que llevaba. Entró el pescador en el barco y pasó una cabra, volvió y pasó otra, tornó a volver y tornó a pasar otra: tenga vuestra merced cuenta con las cabras que el pescador va pasando, porque si se pierde una de la memoria se acabará e
----------------------------------------------------------------------------------------------------
o, en pena de tu mal deseo; mas no me deja usar deste rigor la amistad que te tengo, la cual no consiente que te deje puesto en tan manifiesto peligro de perderte. Y porque claro lo veas, dime, Anselmo: ¿tu no me has dicho que tengo de solicitar a una retirada, persuadir a una ho
----------------------------------------------------------------------------------------------------
che un hombre, que en el traje mostró luego el oficio y cargo que tenía, porque la ropa luenga, con las mangas arrocadas, que vestía

In [7]:
# Caracteres unicos dentro del texto:
vocab = sorted(set(texto))
print(f'{len(vocab)} caracteres unicos.')

89 caracteres unicos.


In [8]:
vocab

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '¡',
 '«',
 '»',
 '¿',
 '̀',
 '́',
 '̃',
 '̈',
 '–',
 '‘',
 '’',
 '“',
 '”']

In [9]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

texto = strip_accents(texto)

for i in range(N):
  inicio = random.randint(0, len(texto)-cata)
  print("-"*100)
  print(texto[inicio:inicio+cata])

----------------------------------------------------------------------------------------------------
e pasaba, y lo mesmo hice a su padre, suplicandole se entretuviese algunos dias y dilatase el darle estado hasta que yo viese lo que Ricardo me queria; el me lo prometio, y ella me lo confirmo con mil juramentos y mil desmayos. Vine, en fin, donde el duque Ricardo estaba. Fui del
----------------------------------------------------------------------------------------------------
manera, con muestras de tanta tristeza, le dijo: Sabete, Sancho, que no es un hombre mas que otro si no hace mas que otro: todas esta borrascas que nos suceden son senales de que presto ha de serenar el tiempo, y han de sucedernos bien las cosas, porque no es posible que el mal n
----------------------------------------------------------------------------------------------------
s, y don Quijote, con muy corteses razones, pidio a los que iban en su guarda fuesen servidos de informalle y decille la causa, o causa

In [10]:
vocab = sorted(set(texto))
print(f'{len(vocab)} caracteres unicos.')

85 caracteres unicos.


# Procesamos el Texto

In [11]:
# Caracteres unicos dentro del texto:
vocab = sorted(set(texto))
print(f'{len(vocab)} caracteres unicos.')

85 caracteres unicos.


In [12]:
vocab

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '¡',
 '«',
 '»',
 '¿',
 '–',
 '‘',
 '’',
 '“',
 '”']

Tal y como realizamos en demostraciones anteriores, vectorizamos el texto, transformado las cadenas de caracteres en una representacion numerica.

tf.keras.layers.StringLookup puede convertir cada caracter en un identificador numerico.

In [13]:
cadenas_ejemplo = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(cadenas_ejemplo, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Creamos una capa de busqueda de identificadores:

In [14]:
crear_ids = tf.keras.layers.StringLookup(vocabulary=list(vocab))

Transformamos los tokens en identificadores:

In [15]:
ids = crear_ids(chars)
ids

<tf.RaggedTensor [[53, 54, 55, 56, 57, 58, 59], [74, 75, 76]]>

Como el objetivo es generar texto, tenemos que invertir la representacion para que lo lea un humano con invert=True:

In [16]:
recuperar_chars = tf.keras.layers.StringLookup(
    vocabulary=crear_ids.get_vocabulary(), invert=True, mask_token=None)

Esta capa recupera el formato de textos:

In [17]:
chars = recuperar_chars(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


Usamos `tf.strings.reduce_join` para unirlos de nuevo en cadenas:

In [18]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [19]:
def texto_de_ids(ids):
  return tf.strings.reduce_join(recuperar_chars(ids), axis=-1)

# La Tarea de Prediccion:

Dado un caracter o una secuencia de caracteres, ¿cual es la siguiente cadena mas probable? Esta es la tarea que realizara el modelo. Le daremos una serie de caracteres y generar un nuevo caracter cada vez. Dado que las redes neuronales recurrentes mantienen un estado interno de los elementos previamente vistos, de los caracteres computados hasta ese momento, ¿cual sera el siguiente caracter?

# Creamos Ejemplos y Objetivos

Dividiremos el texto en secuencias de ejemplo. Cada secuencia contendra una longitud de secuencia dada del texto.

Para cada secuencia, generaremos la secuencia objetivo con la misma longitud de texto, un espacio mas a la derecha.

Utilizamos `tf.data.Dataset.from_tensor_slices` para convertir el vector texto en un flujo de indices.

In [20]:
todos_ids = crear_ids(tf.strings.unicode_split(texto, 'UTF-8'))
todos_ids

<tf.Tensor: shape=(1018041,), dtype=int64, numpy=array([28, 39, 38, ..., 33, 43,  1])>

In [21]:
ids_dataset = tf.data.Dataset.from_tensor_slices(todos_ids)

In [22]:
ids_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [23]:
for ids in ids_dataset.take(15):
    print(recuperar_chars(ids).numpy().decode('UTF-8'))

D
O
N
 
Q
U
I
J
O
T
E
 
D
E
 


In [24]:
longitud_secuencia = 100
ejemplos_por_ciclo = len(texto)//(longitud_secuencia+1)

In [25]:
ejemplos_por_ciclo

10079

El metodo `batch` nos permite convertir estos caracteres en secuencias:

In [26]:
secuencias = ids_dataset.batch(longitud_secuencia+1, drop_remainder=True)

for seq in secuencias.take(1):
  print(recuperar_chars(seq))

tf.Tensor(
[b'D' b'O' b'N' b' ' b'Q' b'U' b'I' b'J' b'O' b'T' b'E' b' ' b'D' b'E'
 b' ' b'L' b'A' b' ' b'M' b'A' b'N' b'C' b'H' b'A' b'\n' b'M' b'i' b'g'
 b'u' b'e' b'l' b' ' b'd' b'e' b' ' b'C' b'e' b'r' b'v' b'a' b'n' b't'
 b'e' b's' b' ' b'S' b'a' b'a' b'v' b'e' b'd' b'r' b'a' b'\n' b'\n' b'P'
 b'R' b'I' b'M' b'E' b'R' b'A' b' ' b'P' b'A' b'R' b'T' b'E' b'\n' b'C'
 b'A' b'P' b'I' b'T' b'U' b'L' b'O' b' ' b'1' b':' b' ' b'Q' b'u' b'e'
 b' ' b't' b'r' b'a' b't' b'a' b' ' b'd' b'e' b' ' b'l' b'a' b' ' b'c'
 b'o' b'n' b'd'], shape=(101,), dtype=string)


Pero no lo vemos en formato texto, lo podemos recuperar a texto:

In [27]:
for seq in secuencias.take(5):
  print(texto_de_ids(seq).numpy())

b'DON QUIJOTE DE LA MANCHA\nMiguel de Cervantes Saavedra\n\nPRIMERA PARTE\nCAPITULO 1: Que trata de la cond'
b'icion y ejercicio del famoso hidalgo D. Quijote de la Mancha\nEn un lugar de la Mancha, de cuyo nombre'
b' no quiero acordarme, no ha mucho tiempo que vivia un hidalgo de los de lanza en astillero, adarga an'
b'tigua, rocin flaco y galgo corredor. Una olla de algo mas vaca que carnero, salpicon las mas noches, '
b'duelos y quebrantos los sabados, lentejas los viernes, algun palomino de anadidura los domingos, cons'


Para el entrenamiento del modelo, necesitamos un set de datos del tipo (entrada, etiqueta). Entrada y etiqueta son secuencias de textos. A cada paso temporal la entrada en el caracter actual y la etiqueta el caracter siguiente. Con esta funcion, duplicamos la secuencia de entrada y la alineamos con su etiqueta correspondiente:

In [28]:
def partir_entrada_objetivo(secuencia):
    entrada = secuencia[:-1]
    etiqueta = secuencia[1:]
    return entrada, etiqueta

In [29]:
partir_entrada_objetivo(list("Universidad Europea"))

(['U',
  'n',
  'i',
  'v',
  'e',
  'r',
  's',
  'i',
  'd',
  'a',
  'd',
  ' ',
  'E',
  'u',
  'r',
  'o',
  'p',
  'e'],
 ['n',
  'i',
  'v',
  'e',
  'r',
  's',
  'i',
  'd',
  'a',
  'd',
  ' ',
  'E',
  'u',
  'r',
  'o',
  'p',
  'e',
  'a'])

In [30]:
dataset = secuencias.map(partir_entrada_objetivo)

In [31]:
for entrada_ejemplo, etiqueta_ejemplo in dataset.take(1):
    print("Entrada :", texto_de_ids(entrada_ejemplo).numpy())
    print("Etiqueta :", texto_de_ids(etiqueta_ejemplo).numpy())

Entrada : b'DON QUIJOTE DE LA MANCHA\nMiguel de Cervantes Saavedra\n\nPRIMERA PARTE\nCAPITULO 1: Que trata de la con'
Etiqueta : b'ON QUIJOTE DE LA MANCHA\nMiguel de Cervantes Saavedra\n\nPRIMERA PARTE\nCAPITULO 1: Que trata de la cond'


### Crear Lotes de Entrenamiento

Generamos lotes de 64 secuencias:

In [32]:
# Tamaño del Lote
Tamaño_Lote = 64

# Tamaño del buffer:
BUFFER = 10000

dataset = (
    dataset
    .shuffle(BUFFER)
    .batch(Tamaño_Lote, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Construccion del Modelo

Definimos una subclase `keras.Model` para generar un modelo con tres capas:



* `tf.keras.layers.Embedding`: la capa de entrada con la busqueda del ID que corresponda a cada caracter;
* `tf.keras.layers.GRU`: La red neural completa con un numero de neuronas.
* `tf.keras.layers.Dense`: La salidad, con una neurona para caracter del vocabulario.

In [33]:
# Vocabulario completo:
tamaño_vocabulario = len(vocab) + 1

# Dimensiones de vectorizacion:
dim_vectorizacion = 128

# Unidades recurrentes.
unidades_rnn = 512

In [34]:
class Modelo(tf.keras.Model):
  def __init__(self, tamaño_vocabulario, dim_vectorizacion, unidades_rnn):
    super().__init__(self)
    self.vectores = tf.keras.layers.Embedding(tamaño_vocabulario,
                                              dim_vectorizacion)
    self.gru = tf.keras.layers.GRU(unidades_rnn,
                                   return_sequences=True,
                                   return_state=True)
    self.densa = tf.keras.layers.Dense(tamaño_vocabulario)

  def call(self, entradas, estados=None, devolver_estado=False, training=False):
    x = entradas
    x = self.vectores(x, training=training)
    if estados is None:
      estados = self.gru.get_initial_state(x)
    x, estados = self.gru(x, initial_state=estados, training=training)
    x = self.densa(x, training=training)

    if devolver_estado:
      return x, estados
    else:
      return x

In [35]:
tamaño_vocabulario

86

In [36]:
modelo = Modelo(
    tamaño_vocabulario,
    dim_vectorizacion,
    unidades_rnn)

Para cada caracter, el modelo busca su vectorizacion, ejecuta la capa de neuronas recurrentes con este vector como entrada, aplica la capa densa para generar le prediccion sobre cual puede ser el siguiente caracter:

![Datos pasando por el modelo](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

## Probar el Modelo

Veamos como se comporta el modelo.

Veremos la forma de la salida:

In [37]:
for lote_entrada, lote_etiqueta in dataset.take(1):
    pred = modelo(lote_entrada)
    print(pred.shape, "# (tamaño del lote, longitud de secuencia, tamaño de vocabulario)")

(64, 100, 86) # (tamaño del lote, longitud de secuencia, tamaño de vocabulario)


La secuencia mide 100 caracteres, podemos ajustar a la longitud que necesitemos.

In [38]:
modelo.summary()

Model: "modelo"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  11008     
                                                                 
 gru (GRU)                   multiple                  986112    
                                                                 
 dense (Dense)               multiple                  44118     
                                                                 
Total params: 1,041,238
Trainable params: 1,041,238
Non-trainable params: 0
_________________________________________________________________


Para obtener prediciones reales desde el modelo, necesitamos extraer de la distribucion de salidas los numeros de indice. Esta distribucion la define el logito (logit) sobre el vocabulario.

In [39]:
cata_indices = tf.random.categorical(pred[10], num_samples=1)
cata_indices = tf.squeeze(cata_indices, axis=-1).numpy()

A cada paso tenemos la prediccion del siguiente caracter en el texto:

In [40]:
cata_indices

array([70, 56, 65, 34, 77, 75, 38, 14, 72, 68, 37,  2, 66, 57, 41, 37, 59,
        2,  0, 83, 35, 29, 81, 81, 41, 84,  8, 17, 70, 84, 53, 75, 15, 34,
       72, 69, 40,  1, 45, 61, 21, 67, 57, 84, 25, 11, 14, 81, 30, 33, 46,
       81, 59, 61, 56, 26, 69, 73, 63, 40, 37, 24, 47,  4, 79, 72, 40, 13,
       40, 49, 72,  5, 64, 81, 20,  8, 15, 49, 21, 75, 50, 40, 76, 51, 17,
       35, 18,  8, 40, 53,  7, 67, 66, 66, 58,  6, 83,  5, 14, 42])

Decodificamos estos indices para saber que caracteres son:



In [41]:
print("Entrada:\n", recuperar_chars(lote_entrada[0]).numpy())
print("Entrada:\n", texto_de_ids(lote_entrada[0]).numpy())

print()

print("Siguiente Caracter Prediccion:\n", recuperar_chars(cata_indices).numpy())
print()
print("Siguiente Caracter Prediccion:\n", texto_de_ids(cata_indices).numpy())

Entrada:
 [b'a' b' ' b'd' b'e' b'c' b'i' b'r' b'l' b'o' b's' b' ' b'c' b'o' b'm'
 b'o' b' ' b'l' b'a' b's' b' ' b'h' b'i' b's' b't' b'o' b'r' b'i' b'a'
 b's' b' ' b'n' b'o' b's' b' ' b'l' b'o' b's' b' ' b'c' b'u' b'e' b'n'
 b't' b'a' b'n' b',' b' ' b's' b'e' b'r' b'i' b'a' b' ' b'n' b'u' b'n'
 b'c' b'a' b' ' b'a' b'c' b'a' b'b' b'a' b'r' b',' b' ' b'y' b' ' b't'
 b'o' b'm' b'a' b'r' b' ' b'l' b'u' b'e' b'g' b'o' b' ' b'l' b'a' b' '
 b'q' b'u' b'e' b' ' b'p' b'a' b'r' b'e' b'c' b'i' b'a' b' ' b'p' b'r'
 b'i' b'n']
Entrada:
 b'a decirlos como las historias nos los cuentan, seria nunca acabar, y tomar luego la que parecia prin'

Siguiente Caracter Prediccion:
 [b's' b'd' b'n' b'J' b'\xc2\xa1' b'y' b'N' b'3' b'u' b'q' b'M' b' ' b'o'
 b'e' b'Q' b'M' b'g' b' ' b'[UNK]' b'\xe2\x80\x99' b'K' b'E'
 b'\xe2\x80\x93' b'\xe2\x80\x93' b'Q' b'\xe2\x80\x9c' b',' b'6' b's'
 b'\xe2\x80\x9c' b'a' b'y' b'4' b'J' b'u' b'r' b'P' b'\n' b'U' b'i' b':'
 b'p' b'e' b'\xe2\x80\x9c' b'A' b'0' b'3' b'\xe2\x80\x93' 

## Entrenar el Modelo


En este punto el modelo es una clasificacion estandard. Dado el estado anterior en la red neuronal recursiva, y la entrada para ese paso, predicimos la clase del siguiente caracter en el texto:

### Definimos el Optimizador y la Funcion de Perdida


La funcion estandar `tf.keras.losses.sparse_categorical_crossentropy` funciona ya que se aplica a la ultima capa de lared.

El modelo devuelve el ["logito"](https://en.wikipedia.org/wiki/Logit) y por lo tanto debemos utilizar la opcion `from_logits`.


In [42]:
perdida = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [43]:
perdida_lote_ejemplo = perdida(lote_entrada, pred)
perdida_media = perdida_lote_ejemplo.numpy().mean()
print("Forma de la Prediccion: ", pred.shape, " # (tamaño lote, longitud secuencia, tamaño vocabulario)")
print("Perdida Media:        ", perdida_media)

Forma de la Prediccion:  (64, 100, 86)  # (tamaño lote, longitud secuencia, tamaño vocabulario)
Perdida Media:         4.4528923


Un modelo nuevo no sabe lo que hace, todas las salidas del logito son similares. Podemos confirmarlo con el exponente de la perdida media y veremos que se parece al numero de elementos del vocabulario. El modelo sabe que sus resutaldos no pueden ser buenos con esta perdida media tan alta:

In [44]:
tf.exp(perdida_media).numpy()

85.87496

Compilamos el modelo con su optimizador y su funcion de perdida:

In [45]:
modelo.compile(optimizer='adam', loss=perdida)

### Configurar Puntos de Recuperacion (Checkpoints)

Usamos un objeto de recuperacion `tf.keras.callbacks.ModelCheckpoint` para asegurarnos que guardamos el modelo a cada epoca:

In [46]:
# Directorio para almacenar los puntos de recuperacion:
modelos_dir = './modelos_entrenados'
# Nombre de los archivos:
prefijo_archivos = os.path.join(modelos_dir, "Epoca_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=prefijo_archivos,
    save_weights_only=True)

### Entrenamos el Modelo

Aqui ya podemos entrenar el modelo durante tantas epocas como nos podamos permitir. Empecemos con 10 para que no sea muy largo y observar como se comporta el modelo con poco entrenamiento. Generalmente hacemos esto para probar que el modelo es sintacticamente correcto, aunque tenga poca capacidad de prediccion:

In [47]:
EPOCHS = 10

In [48]:
history = modelo.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
157/157 [==============================] - 12s 55ms/step - loss: 2.6471
Epoch 2/10
157/157 [==============================] - 10s 54ms/step - loss: 1.9843
Epoch 3/10
157/157 [==============================] - 10s 54ms/step - loss: 1.7882
Epoch 4/10
157/157 [==============================] - 10s 54ms/step - loss: 1.6284
Epoch 5/10
157/157 [==============================] - 10s 54ms/step - loss: 1.5074
Epoch 6/10
157/157 [==============================] - 10s 54ms/step - loss: 1.4226
Epoch 7/10
157/157 [==============================] - 11s 59ms/step - loss: 1.3623
Epoch 8/10
157/157 [==============================] - 11s 54ms/step - loss: 1.3173
Epoch 9/10
157/157 [==============================] - 10s 54ms/step - loss: 1.2810
Epoch 10/10
157/157 [==============================] - 10s 53ms/step - loss: 1.2514


## Generacion de Textos

La manera mas simple de generar textos con este modelo es ejecutarlo en un ciclo. En cada ejecucion podemos extraer el estado interno del modelo mientras es ejecutado:
![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Cada vez que llamamos al modelo pasamos algun estado interno. El modelo devuelve la prediccion para el siguiente caracter y su nuevo estado (estado del modelo). Pasamos la prediccion de nuevo por el modelo para generar un nuevo caracter, y asi sucesivamente:

La siguiente clase genera un ciclo de prediccion:

In [49]:
class ModeloUnPaso(tf.keras.Model):
  def __init__(self, modelo,
               caracteres_de_ids,
               ids_de_caracteres,
               temperatura=1.0):
    super().__init__()
    self.temperatura = temperatura
    self.modelo = modelo
    self.caracteres_de_ids = caracteres_de_ids
    self.ids_de_caracteres = ids_de_caracteres

    # Creamos una mascara para evitar
    # la generacion de "[UNK]", caracteres desconocidos fuera de vocabulario.
    saltar_ids = self.ids_de_caracteres(['[UNK]'])[:, None]
    mascara = tf.SparseTensor(
        # -inf para cada indice erroneo.
        values=[-float('inf')]*len(saltar_ids),
        indices=saltar_ids,
        # Obtener mismo tamaño de vocabulario:
        dense_shape=[len(ids_de_caracteres.get_vocabulary())])
    self.mascara_prediccion = tf.sparse.to_dense(mascara)

  @tf.function
  def generar_paso(self, entradas, estados=None):
    # Convertir IDs a caracteres.
    caracteres_entrada = tf.strings.unicode_split(entradas, 'UTF-8')
    ids_entradas = self.ids_de_caracteres(caracteres_entrada).to_tensor()

    # Ejecutar el modelo.
    logito_prediccion, estados = self.modelo(entradas=ids_entradas,
                                            estados=estados,
                                            devolver_estado=True)
    # Usamos solamente la ultima prediccion:
    logito_prediccion = logito_prediccion[:, -1, :]
    logito_prediccion = logito_prediccion/self.temperatura
    # Aplicamos la mascara para evitar la generacion de caracteres desconocidos:
    logito_prediccion = logito_prediccion + self.mascara_prediccion

    # Generamos identificadores del los logitos:
    ids_prediccion = tf.random.categorical(logito_prediccion, num_samples=1)
    ids_prediccion = tf.squeeze(ids_prediccion, axis=-1)

    # Converitmos los id en caracteres:
    caracteres_prediccion = self.caracteres_de_ids(ids_prediccion)

    # Devolvemos la prediccion y el estado del modelo:
    return caracteres_prediccion, estados

In [50]:
modelo_un_paso = ModeloUnPaso(modelo, recuperar_chars, crear_ids)

Ejecutamos el modelo en un ciclo para generar un texto. Mirando el texto generado, veremos que el modelo mas o menos puede colocar las mayuculas bien, hace parrafos, imita un poco a Cervantes en su vocabulario. Con los pocos ciclos o epocas de entrenamiento todavia no puede genear frases coherentes:

In [51]:
inicio = time.time()
estados = None
siguiente_caracter = tf.constant(['Sancho'])
resultado = [siguiente_caracter]

for n in range(1000):
  # Si queremos ver cada caracter generado:
  # print(siguiente_caracter)
  siguiente_caracter, estados = modelo_un_paso.generar_paso(siguiente_caracter, 
                                                            estados=estados)
  resultado.append(siguiente_caracter)

resultado = tf.strings.join(resultado)
fin = time.time()
print("-------------------------------")
print(resultado[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nTiempo de Ejecucion:', fin - inicio)

-------------------------------
Sancho de un femucion, o menoras le refendido en cardenio en decir:
Sincho, vemos en el
por que todo vengamiento. Quedarse despues de honested que gente todo el dio adelante, que acaso del ro, que, contines el suelo dos son Bella ellos la vuestra merced, no digo yo quedose baja, el oyero y obligo a mejor liberalmente su padre y en consiguieron dia de toda la pende no pudiese, los mas, sin arnoy dame yo que habia en dos amisos.
¿Es o compuebo dos imaginos, respondio Sancho.
Biera sobre ellos:
que dobia hacer lo mismo, por esta vida, porque mas eran eternas de los mesmos caso resiturosas como un sabio tantos comodiaron hasta vuelto aplanta que pasada, pareciendo encompanero de reir de la saber aquella estando asi como ya tan sujese a los caballeros desposos», que te hubo posible blanca el tiempo de Camila, por quien donde se daban barcayaron cuantos; mas llego a los representad y al findieronla, como el de enclubirme vogo este hombre de rama.
Y parecen que

La manera mas simple de mejorar los resultados es entrenar mas el modelo.

Podemos cambiar tambien los caracteres de arranque, añadir mas capas recurrentes o ajustar el parametro de temperatura para generar predicciones mas o menos aleatorias.

In [52]:
EPOCHS = 20
history = modelo.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
157/157 [==============================] - 10s 54ms/step - loss: 1.2244
Epoch 2/20
157/157 [==============================] - 10s 54ms/step - loss: 1.2015
Epoch 3/20
157/157 [==============================] - 10s 55ms/step - loss: 1.1808
Epoch 4/20
157/157 [==============================] - 10s 54ms/step - loss: 1.1608
Epoch 5/20
157/157 [==============================] - 10s 54ms/step - loss: 1.1426
Epoch 6/20
157/157 [==============================] - 10s 54ms/step - loss: 1.1251
Epoch 7/20
157/157 [==============================] - 10s 54ms/step - loss: 1.1078
Epoch 8/20
157/157 [==============================] - 10s 54ms/step - loss: 1.0908
Epoch 9/20
157/157 [==============================] - 10s 54ms/step - loss: 1.0745
Epoch 10/20
157/157 [==============================] - 10s 54ms/step - loss: 1.0581
Epoch 11/20
157/157 [==============================] - 10s 54ms/step - loss: 1.0426
Epoch 12/20
157/157 [==============================] - 10s 54ms/step - loss: 1.0265
E

Si queremos generar texto de manera mas rapida, podemos hacerlo por lotes. En este ejemplo generamos 5 salidas de manera mas eficiente:

In [53]:
inicio = time.time()
estados = None
siguiente_caracter = tf.constant(
    ['Sancho ',
     'Rocinante ',
     'Dulcinea ',
     'Hector es el mejor ',
     'gigante'])

resultado = [siguiente_caracter]

for n in range(1000):
  siguiente_caracter, estados = modelo_un_paso.generar_paso(siguiente_caracter,
                                                            estados=estados)
  resultado.append(siguiente_caracter)

resultado = tf.strings.join(resultado)
fin = time.time()
print(resultado, '\n\n' + '_'*80)
print('\nTiempo de Ejecucion:', fin - inicio)

tf.Tensor(
[b'Sancho . Y de una ya no quise determina se que deja en la hacienda, tencionesa y menos suelen tan suspenos. No conocieron cuantos palaban. El caminante quiso roda, porque se lo falte en el innumerio con dos dias de pastores del que llevaba algas que elli no por el suceso en Fernardo, continuo yo de confirmar otras al reves? \xc2\xa1Oh Ansilmos? Otros andantes pasaron entonces, dijo Sancho, pero Zoraida son juntos, contento que los otros incomodidades son algun del mal instante por donde los dejos contra tomagan los montes y todos contra perjuicios. Pero, con nuevo y pena de la almera podia haga de carecerlo y estado como si de cuanto el no saber lo que Sancho hubiese cuerto alguno premetio en su palabra, mas, que no quise derrubirse de ligarme de don Femando y acogida, habiendome gusto de persuadirme la verdadera amistad que le tengan y viniene a oscuras, huyos, que yo entendio el suceso de la nopa, que alli otra cosa se habia enfermedo y labiado, en la ciudad de Anselmo;

## Exportando el Generador


Este modelo se puede [guardar y cargar](https://www.tensorflow.org/guide/saved_model), utilizandolo en cualquier instancia donde un `tf.saved_model` se aceptable.

In [54]:
tf.saved_model.save(modelo_un_paso, 'un_paso')
un_paso_reloaded = tf.saved_model.load('un_paso')

INFO:tensorflow:Assets written to: un_paso/assets


INFO:tensorflow:Assets written to: un_paso/assets


In [55]:
estados = None
siguiente_caracter = tf.constant(['Quijote '])
resultado = [siguiente_caracter]

for n in range(100):
  siguiente_caracter, estados = un_paso_reloaded.generar_paso(siguiente_caracter,
                                                              estados=estados)
  resultado.append(siguiente_caracter)

print(tf.strings.join(resultado)[0].numpy().decode("utf-8"))

Quijote deseaba, que pues, era un desden que es una bacia y roman en mentirosa. Digo. -pregunto Vivos –dijo 


## Entrenamiento Personalizado

Los procesos de entrenamiento utilizados son simples y no nos otorgan mucho control. Fuerza que las predicciones incorrectas vuelvan al modelo, el modelo nunca aprende a corregir sus propios errores.

Podemos implementar un ciclo de entranamiento para estabilizar las predicciones del modelo. La parte mas importante es la parte donde definimos un paso de entrenamiento.

Usamos `tf.GradientTape` para inspeccionar los gradients. [Guia de ejecucion.](https://www.tensorflow.org/guide/eager).

El proceso basico es:

1. Ejecutar el modelo y calcular la perdida `tf.GradientTape`.
2. Calcular las actualizaciones y aplicarlas al modelo utilizando el optimizador.

In [56]:
class EntrenamientoPersonalizado(Modelo):
  @tf.function
  def train_step(self, entradas):
      entradas, etiquetas = entradas
      with tf.GradientTape() as cinta:
          predicciones = self(entradas, training=True)
          perdida = self.loss(etiquetas, predicciones)
      grads = cinta.gradient(perdida, modelo.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, modelo.trainable_variables))

      return {'Perdida': perdida}

Hemos reimplementado el metodo `train_step` siguiendo [las convenciones de Keras para  `train_step`](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). Es totalemente opcional, nos permite cambiar el comportamiento del paso de entrenamiento usando metodos de Keras (.fit).

In [57]:
modelo = EntrenamientoPersonalizado(
    tamaño_vocabulario=len(crear_ids.get_vocabulary()),
    dim_vectorizacion=dim_vectorizacion,
    unidades_rnn=unidades_rnn)

In [58]:
modelo.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [59]:
modelo.fit(dataset, epochs=1)

157/157 [==============================] - 12s 54ms/step - Perdida: 2.5125


Si necesitamos mas control podemos escribir nuestro ciclo de entrenamiento completo propio:

In [60]:
EPOCHS = 10

media = tf.metrics.Mean()

for e in range(EPOCHS):
    inicio = time.time()

    media.reset_states()
    for (lote_n, (inp, objetivo)) in enumerate(dataset):
        logs = modelo.train_step([inp, objetivo])
        media.update_state(logs['Perdida'])

        if lote_n % 50 == 0:
            mensaje_tipo = f"Epoca {e+1} Lote {lote_n} Perdida {logs['Perdida']:.4f}"
            print(mensaje_tipo)

    # Grabamos cada 5 epocas
    if (e + 1) % 5 == 0:
        modelo.save_weights(prefijo_archivos.format(epoch=e))

    print()
    print(f'Epoca {e+1} perdida: {media.result().numpy():.4f}')
    print(f'Tiempo para una epoca {time.time() - inicio:.2f} s')
    print("_"*80)

modelo.save_weights(prefijo_archivos.format(epoch=e))

Epoca 1 Lote 0 Perdida 2.0679
Epoca 1 Lote 50 Perdida 1.9867
Epoca 1 Lote 100 Perdida 1.9166
Epoca 1 Lote 150 Perdida 1.8711

Epoca 1 perdida: 1.9550
Tiempo para una epoca 10.22 s
________________________________________________________________________________
Epoca 2 Lote 0 Perdida 1.8365
Epoca 2 Lote 50 Perdida 1.7621
Epoca 2 Lote 100 Perdida 1.7236
Epoca 2 Lote 150 Perdida 1.7173

Epoca 2 perdida: 1.7601
Tiempo para una epoca 9.50 s
________________________________________________________________________________
Epoca 3 Lote 0 Perdida 1.6725
Epoca 3 Lote 50 Perdida 1.6235
Epoca 3 Lote 100 Perdida 1.6006
Epoca 3 Lote 150 Perdida 1.5522

Epoca 3 perdida: 1.6019
Tiempo para una epoca 9.43 s
________________________________________________________________________________
Epoca 4 Lote 0 Perdida 1.5258
Epoca 4 Lote 50 Perdida 1.5348
Epoca 4 Lote 100 Perdida 1.4494
Epoca 4 Lote 150 Perdida 1.4397

Epoca 4 perdida: 1.4846
Tiempo para una epoca 9.39 s
________________________________________